# New Code For LiLab
因为需要针对实验室的实际情况设计FLPo和Cre的探针
1-调用NCBI的检索模式发生改变
2-输入的gene list矩阵格式也需要改变

## 1 设置运行环境

In [115]:
# basci env
import os
from pathlib import Path
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# dir
# DATASET_DIR = Path('/mnt/f/spatial_data/probe')
DATASET_DIR = Path('./probe')
RUNID = '20250612'
workdir = DATASET_DIR / RUNID
os.makedirs(workdir, exist_ok=True)
organism = 'mouse'

## 2 创建result文件夹

In [2]:
# create results dir
current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)
output = os.path.join(workdir, 'results', formatted_time+'_ensembl')
bds_candidate_dir = os.path.join(output, "bds_candidate")
os.makedirs(output, exist_ok=True)
os.makedirs(bds_candidate_dir, exist_ok=True)

# file name variables
bds_candidate_file_suffix = "_bds_candidate.fasta"
combined_bds_candidates_file = "total_bds_candidate.fasta"
combined_bds_candidates_blast_file = 'total_bds_candidate_blast.fasta'
# bds_candidate_num_file = "bds_candidate_num.json"
bds_candidate_num_file = "merge.json"
blast_results_file = "blast_results.xml"

## 3 按照Huang Lab策略获取传统基因json文件

In [3]:
workdir

PosixPath('probe/20250612')

In [4]:
# ## 这个步骤可以选择直接将基因读入,其实可以直接输入基因名
# gene_info = pd.read_excel(os.path.join(workdir, "gene_info.xlsx"))
gene_info = pd.read_table(os.path.join(workdir, "gene_list.txt"))
# 如果需要转换大小写
gene_info['gene'] = gene_info['gene_name']
if organism == 'mouse': gene_info['gene'] = gene_info['gene'].str.capitalize()
elif organism == 'human': gene_info['gene'] = gene_info['gene'].str.upper()
gene_list = [_.strip() for _ in gene_info['gene'].unique() if _!=0]
print(len(gene_list))
gene_info.head()

13


,gene_name,gene
0,Gfap,Gfap
1,Aif1,Aif1
2,Cartpt,Cartpt
3,Vipr2,Vipr2
4,Glp1r,Glp1r


In [ ]:
from lib.database_interaction import ensembl_name_to_seqs
import time
max_trial = 3
sequences_of_all = dict()
error_messages = {gene: [] for gene in gene_list}

with tqdm(total=len(gene_list), desc="Retriving_sequences", position=0) as pbar_total:
    for gene in gene_list:
        sequences_of_all[gene] = {}
        trial_success = False
        
        # Reset the trial progress bar for each gene
        for trial in range(1, max_trial+1):  # Retrying up to 3 times
            try:
                # Attempt to retrieve sequences
                sequences_of_all[gene] = ensembl_name_to_seqs(gene=gene, species=organism, seq_type='cds', tqdm_args={'position': 1,'leave': False})
                trial_success = True
                break
            except Exception as e:
                time.sleep(1)

        if not trial_success:
            error_messages[gene].append(f"Failed to retrieve sequences for {gene} after {max_trial} attempts.")

        pbar_total.update(1)  # Update the main progress bar after each gene

for gene, messages in error_messages.items():
    for message in messages:
        print(message)

with open(os.path.join(output, 'sequence_of_all.json'), 'w') as file: json.dump(sequences_of_all, file)

In [6]:
with open(os.path.join(output, 'sequence_of_all.json'), 'r') as file: sequences_of_all = json.load(file)

isoforms = {}
for gene, isoforms_tmp in sequences_of_all.items():
    shortest_isoform = None
    min_length = 10**6
    max_length = 0
    for isoform in isoforms_tmp:
        try: isoform_name = isoform['external_name']
        except: continue
        if len(isoform['seq']) < min_length and gene.upper() in isoform_name.upper():
            min_length = len(isoform['seq'])
            shortest_isoform = isoform
    if shortest_isoform:
        isoforms[gene] = shortest_isoform
with open(os.path.join(output, 'shortest_isoforms.json'), 'w') as file: json.dump(isoforms, file)

## 获取LiLab定制 FLPo 和 Cre json文件

查看读取文件

In [7]:
gene_info = pd.read_excel(os.path.join(workdir, "gene_genbank_id20250612_3genes.xlsx"))
print(gene_info)
print(gene_info['gene_name'])

     genbank_id                                    product  \
0  NM_001131020  glial fibrillary acidic protein isoform 1   
1    KX151729.1                                       FLPo   
2    EU693012.1                            cre recombinase   

               gene_name  
0                   Gfap  
1             FLPo_LiLab  
2  cre recombinase_LiLab  
0                     Gfap
1               FLPo_LiLab
2    cre recombinase_LiLab
Name: gene_name, dtype: object


检查查询策略

In [8]:
# 函数
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "1418767067@qq.com"  # 设置邮箱
Entrez.api_key = '010eacb785458478918b0cb14bea9f9df609'
# 获取完整GenBank记录
handle = Entrez.efetch(
    db="nuccore", 
    id="KX151729.1", 
    rettype="gb"
)
record = SeqIO.read(handle, "genbank")

# 定位FLPo基因
for feature in record.features:
    if feature.type == "CDS" and "FLPo" in str(feature.qualifiers.get("product", "")):
        start = feature.location.start
        end = feature.location.end
        gene_seq = record.seq[start:end]
        print(f"FLPo基因位置: {start}-{end}")
        print(f"核苷酸序列:\n{gene_seq}")
        break
else:
    print("未找到FLPo基因")

FLPo基因位置: 1901-3200
核苷酸序列:
ATGGCTCCTAAGAAGAAGAGGAAGGTGATGAGCCAGTTCGACATCCTGTGCAAGACCCCCCCCAAGGTGCTGGTGCGGCAGTTCGTGGAGAGATTCGAGAGGCCCAGCGGCGAGAAGATCGCCAGCTGTGCCGCCGAGCTGACCTACCTGTGCTGGATGATCACCCACAACGGCACCGCCATCAAGAGGGCCACCTTCATGAGCTACAACACCATCATCAGCAACAGCCTGAGCTTCGACATCGTGAACAAGAGCCTGCAGTTCAAGTACAAGACCCAGAAGGCCACCATCCTGGAGGCCAGCCTGAAGAAGCTGATCCCCGCCTGGGAGTTCACCATCATCCCTTACAACGGCCAGAAGCACCAGAGCGACATCACCGACATCGTGTCCAGCCTGCAGCTGCAGTTCGAGAGCAGCGAGGAGGCCGACAAGGGCAACAGCCACAGCAAGAAGATGCTGAAGGCCCTGCTGTCCGAGGGCGAGAGCATCTGGGAGATCACCGAGAAGATCCTGAACAGCTTCGAGTACACCAGCAGGTTCACCAAGACCAAGACCCTGTACCAGTTCCTGTTCCTGGCCACATTCATCAACTGCGGCAGGTTCAGCGACATCAAGAACGTGGACCCCAAGAGCTTCAAGCTGGTGCAGAACAAGTACCTGGGCGTGATCATTCAGTGCCTGGTGACCGAGACCAAGACAAGCGTGTCCAGGCACATCTACTTTTTCAGCGCCAGAGGCAGGATCGACCCCCTGGTGTACCTGGACGAGTTCCTGAGGAACAGCGAGCCCGTGCTGAAGAGAGTGAACAGGACCGGCAACAGCAGCAGCAACAAGCAGGAGTACCAGCTGCTGAAGGACAACCTGGTGCGCAGCTACAACAAGGCCCTGAAGAAGAACGCCCCCTACCCCATCTTCGCTATCAAGAACGGCCCTAAGAGCCACATCGGCAGGCACCTGATGACCAGCTTTCTGA

## 展示输入矩阵,product列需要自行查询

## 测试一下genbank检索功能使否可用 用于检索FLPo 和 Cre 

## 检索FLPo 和 Cre  输出结果为json文件 回头与其他基因合并

In [9]:
# 查询参数
# genbank_id = "NM_001131020.1"  # 替换为实际的 GenBank ID
# gene_name = "glial fibrillary acidic proteins isoform 1"        # 替换为要查询的基因名称

In [10]:
output

'probe/20250612/results/20250614_060230_ensembl'

In [11]:
workdir

PosixPath('probe/20250612')

### 批量查看FLPo 和 Cre 序列的函数

In [12]:
# 函数：获取基因序列（支持重试）
def get_gene_sequence(genbank_id, gene_name, external_name,max_retries=3):
    """
    从GenBank获取指定基因的序列（带重试机制）
    
    参数:
        genbank_id: GenBank ID
        gene_name: 基因名称
        max_retries: 最大重试次数
    
    返回:
        dict: 包含序列信息和状态的字典
    """
    for attempt in range(1, max_retries + 1):
        try:
            # 获取GenBank记录
            handle = Entrez.efetch(db="nuccore", id=genbank_id, rettype="gb")
            record = SeqIO.read(handle, "genbank")
            handle.close()
            
            # 遍历特征寻找目标基因
            for feature in record.features:
                if feature.type == "CDS":
                    product = feature.qualifiers.get("product", [""])[0]
                    if gene_name.lower() in product.lower():
                        start = int(feature.location.start)
                        end = int(feature.location.end)
                        sequence = str(record.seq[start:end])
                        return {
                            "status": "success",
                            "seq": sequence,
                            "external_name":external_name,
                            "genbank_id": genbank_id,
                            "gene_name": gene_name,
                            # "biotype":"protein_coding",
                            "length": len(sequence),
                            "start": start,
                            "end": end
                        }
            
            # 如果未找到匹配的基因
            return {
                "status": "not_found",
                "message": f"Gene '{gene_name}' not found in {genbank_id}"
            }
            
        except Exception as e:
            error_msg = f"Attempt {attempt} failed for {gene_name}({genbank_id}): {str(e)}"
            if attempt < max_retries:
                time.sleep(1)  # 重试前等待
                continue
            return {
                "status": "error",
                "message": error_msg
            }

In [13]:
# 主处理函数（与Ensembl代码格式一致）
def process_genes(gene_info, output_dir):
    """
    批量处理基因数据（参考Ensembl代码格式）
    
    参数:
        gene_info: 包含基因信息的DataFrame
        output_dir: 输出目录
    """
    sequences_of_all = {}
    error_messages = {gene: [] for gene in gene_info['gene_name']}
    
    # 带进度条的处理循环
    with tqdm(total=len(gene_info), desc="Retrieving sequences", position=0) as pbar_total:
        for idx, row in gene_info.iterrows():
            genbank_id = row['genbank_id']
            gene_name = row['product'] ## 可能需要修改的地方
            external_name=row['gene_name']
            
            # 获取基因序列
            result = get_gene_sequence(genbank_id, gene_name,external_name)
            
            # 处理结果
            if result['status'] == "success":
                sequences_of_all[gene_name] = {
                    "seq": result['seq'],
                    "external_name":result['external_name'],
                    "id": genbank_id,
                    "biotype":"protein_coding",
                    "length": result['length'],
                    "location": f"{result['start']}-{result['end']}"
                }
            else:
                error_messages[gene_name].append(result['message'])
                
            pbar_total.update(1)  # 更新主进度条
    
    # 输出错误信息
    for gene, messages in error_messages.items():
        if messages:
            for message in messages:
                print(f"ERROR: {message}")
    
    # 保存JSON结果
    output_file = os.path.join(output_dir, 'sequence_of_all_ex.json')
    with open(output_file, 'w') as f:
        json.dump(sequences_of_all, f, indent=2)
    
    print(f"\n处理完成！结果已保存至: {output_file}")

### 调用上述函数 输出json文件

In [14]:
##算是可用 输出的json文件存在问题
import os
import json
import time
import pandas as pd
from Bio import Entrez, SeqIO
from tqdm import tqdm
# from lib.lilab_code import  get_gene_sequence, process_genes
# 设置Entrez API凭证
Entrez.email = "1418767067@qq.com"
Entrez.api_key = '010eacb785458478918b0cb14bea9f9df609'

# 创建输出目录
output_dir = output
os.makedirs(output_dir, exist_ok=True)

# 执行处理
process_genes(gene_info, output_dir)

Retrieving sequences: 100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


处理完成！结果已保存至: probe/20250612/results/20250614_060230_ensembl/sequence_of_all_ex.json


### 合并2个json文件

In [26]:
import json

# 读取第一个文件
with open(os.path.join(output,'shortest_isoforms.json'), 'r') as f1:
    dict1 = json.load(f1)   # 假设file1.json是一个字典

# 读取第二个文件
with open(os.path.join(output,'sequence_of_all_ex.json'), 'r') as f2:
    dict2 = json.load(f2)   # 假设file2.json是一个字典

# 合并：将dict2合并到dict1，相同键会被覆盖
# 注意：update会直接修改dict1，为了避免改动原数据，我们可以先复制
merged_dict = dict1.copy()
merged_dict.update(dict2)

# 或者使用字典解包（合并两个字典，相同键后面的值会覆盖前面的）
# merged_dict = {**dict1, **dict2}

# 将合并后的字典写入文件
with open(os.path.join(output,'merge.json'), 'w') as out_file:
    json.dump(merged_dict, out_file, indent=2)

In [16]:
output

'probe/20250612/results/20250614_060230_ensembl'

In [17]:
workdir

PosixPath('probe/20250612')

## 读取新的json文件为isoforms

In [27]:
import json

# 读取 JSON 文件
with open(os.path.join(output,'merge.json'), 'r') as f:
    data = json.load(f)

# 使用数据
print(data)

{'Gfap': {'ccdsid': 'CCDS48950.1', 'feature_type': 'transcript', 'start': 102780992, 'biotype': 'protein_coding', 'id': 'ENSMUST00000077902', 'logic_name': 'ensembl_mus_musculus', 'end': 102788026, 'tag': ['gencode_basic'], 'transcript_id': 'ENSMUST00000077902', 'transcript_support_level': '1 (assigned to previous version 4)', 'description': None, 'external_name': 'Gfap-202', 'source': 'ensembl', 'version': 5, 'seq_region_name': '11', 'strand': -1, 'assembly_name': 'GRCm39', 'Parent': 'ENSMUSG00000020932', 'is_canonical': 0, 'seq': 'ATGGAGCGGAGACGCATCACCTCTGCGCGCCGCTCCTATGCCTCCGAGACGGTGGTCAGGGGCCTCGGTCCTAGTCGACAACTGGGTACCATGCCACGCTTCTCCTTGTCTCGAATGACTCCTCCACTCCCTGCCAGGGTGGACTTCTCCCTGGCCGGGGCGCTCAATGCTGGCTTCAAGGAGACACGGGCGAGCGAGCGTGCAGAGATGATGGAGCTCAATGACCGCTTTGCTAGCTACATCGAGAAGGTCCGCTTCCTGGAACAGCAAAACAAGGCGCTGGCAGCTGAACTGAACCAGCTTCGAGCCAAGGAGCCCACCAAACTGGCTGATGTCTACCAGGCGGAGCTTCGGGAGCTGCGGCTGCGGCTGGACCAGCTTACGGCCAACAGTGCCCGGCTGGAGGTGGAGAGGGACAACTTTGCACAGGACCTCGGCACCCTGAGGCAGAAGCTCCAAGA

## 查找绑定位点 获取blast文件

In [ ]:
# isoforms

In [ ]:
# data.items() 
# data

In [28]:
%reload_ext autoreload
%autoreload 2

# set lib auto reload in jupyter notebook
from lib.search_binding import position_search, optimize_subsequence, seq_minus

# Initiation of array
binding_site_entry = [
    "accession", "gene_name", "mol_type", "organism",
    "pos", "plp_bds", "plp_Tm","plp_bds3'", "plp_bds5'", "plp_Tm3'", "plp_Tm5'", "mfe", "wanted"] ## 构建表头
alignment_entry = ["align_num", "align_accession", "align_descrip", "plus/minus"]
BDS_INFO = pd.DataFrame(columns=binding_site_entry+alignment_entry)

# Search binding sites on mRNA sequence
pre_binding_num = {}

# initialization of file
with open(os.path.join(output, combined_bds_candidates_file), "w") as handle: handle.write("")
with open(os.path.join(output, combined_bds_candidates_blast_file), "w") as f: f.write("")

# for desc, info in tqdm(isoforms.items(), desc="Searching_binding_sites", position=0):
for desc, info in tqdm(data.items(), desc="Searching_binding_sites", position=0):
    seq = info['seq']
    if 'N' in seq: seq = seq.replace('N', '')
    try: gene_name= info['external_name']
    except: gene_name = desc

    id = info['id']
    mol_type = info['biotype']
    
    pos_info = position_search(
        seq, gene=gene_name,
        BDS_len=40, BDS_num=100, min_gap=0, better_gap=40, pin_gap=0.05, 
        G_min=0.25, G_max=0.7, G_consecutive=5, Tm_low=48, Tm_high=60, 
        verbose_pos=1, leave=False, warn=False) #设置筛选条件
    
    record_list = []
    for i, pre_binding_tmp in enumerate([_['plp_bds'] for _ in pos_info]):
        record_list.append(
            SeqRecord(Seq(pre_binding_tmp), id="bds_candidate" + str(i), 
                      description="|".join([id, gene_name, organism, mol_type])))

    # add information about binding sites to FOI
    add = pd.DataFrame(pos_info)
    add['accession'] = id
    add['gene_name'] = gene_name
    add['mol_type'] = mol_type
    add['organism'] = organism
    BDS_INFO = pd.concat([BDS_INFO, add], ignore_index=True)

    file_out = os.path.join(bds_candidate_dir, gene_name + bds_candidate_file_suffix)
    print(file_out)
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list: SeqIO.write(new_record, f, "fasta")
    with open(os.path.join(output, combined_bds_candidates_file), "a") as handle:
        for new_record in record_list: SeqIO.write(new_record, handle, "fasta")
    with open(os.path.join(output, combined_bds_candidates_blast_file), "a") as handle:
        for new_record in record_list: 
            blast_seq = str(new_record.seq)
            blast_seq = blast_seq[len(blast_seq)//2-16:len(blast_seq)//2+16] #提取中心32 nt生成BLAST专用文件
            new_record = SeqRecord(Seq(blast_seq), id=new_record.id, description=new_record.description)
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(pos_info)

with open(os.path.join(output, bds_candidate_num_file), "w") as f: json.dump(pre_binding_num, f)

Gfap-202:  47%|████▋     | 529/1119 [00:00<00:00, 5272.96it/s]
                                                              /tmp/ipykernel_572296/871504708.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  BDS_INFO = pd.concat([BDS_INFO, add], ignore_index=True)
Searching_binding_sites:   6%|▋         | 1/16 [00:00<00:03,  4.26it/s]

probe/20250612/results/20250614_060230_ensembl/bds_candidate/Gfap-202_bds_candidate.fasta



Aif1-204:   0%|          | 0/307 [00:00<?, ?it/s]
                                                 

probe/20250612/results/20250614_060230_ensembl/bds_candidate/Aif1-204_bds_candidate.fasta



Searching_binding_sites:  19%|█▉        | 3/16 [00:00<00:01, 10.19it/s]

...

probe/20250612/results/20250614_060230_ensembl/bds_candidate/Gfap_bds_candidate.fasta



Searching_binding_sites:  94%|█████████▍| 15/16 [00:01<00:00,  8.16it/s]

probe/20250612/results/20250614_060230_ensembl/bds_candidate/FLPo_LiLab_bds_candidate.fasta



Searching_binding_sites: 100%|██████████| 16/16 [00:01<00:00,  8.14it/s]  

probe/20250612/results/20250614_060230_ensembl/bds_candidate/cre recombinase_LiLab_bds_candidate.fasta


##需要导出fasta文件，用blast网站处理：
##https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&BLAST_SPEC=&LINK_LOC=blasttab&LAST_PAGE=tblastn
##输出后修改文件名为blast_results.xml

## NCBI 手动blast 之后

In [30]:
# Extract interested information from blast_results 提取blast的结果
from Bio.Blast import NCBIXML
align_num = []
# read the id/plus-minus part/align_num
with open(os.path.join(output, blast_results_file), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        BDS_INFO.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)
        # add align_descrip to df
        BDS_INFO.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)
        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]
        # add plus/minus to df
        try: BDS_INFO.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except: BDS_INFO.loc[loca, "plus/minus"] = pd.NA
        loca += 1
BDS_INFO["align_num"] = align_num

### 额外添加基因到gene_list

In [37]:
gene_list.append('FLPo_LiLab') 
gene_list.append('cre recombinase_LiLab') 
gene_list

['Gfap',
 'Aif1',
 'Cartpt',
 'Vipr2',
 'Glp1r',
 'Il6ra',
 'Oprm1',
 'Pdyn',
 'Prkcd',
 'Sst',
 'Gapdh',
 'Slc17a7',
 'Gad1',
 'FLPo_LiLab',
 'FLPo_LiLab',
 'cre recombinase_LiLab']

In [38]:
## 选择结合位点
import re

def adjust_gene_name(gene_name, gene_list):
    gene_list = [x.upper() for x in gene_list]
    match = re.search(r'(.+)-(\d+)$', gene_name)
    if match:
        base_gene_name = match.group(1)
        if base_gene_name.upper() in gene_list or gene_name.upper() in gene_list: return base_gene_name
        else: return gene_name
    else: return gene_name
## 原文输出probes_wanted.xlsx文档 因为.xlsx的文档在jupyter lab中很难打开
## 于是尝试修改输出csv文件,成功
BDS_INFO["wanted"] = [True] * len(BDS_INFO)
verbose = True
# select by specifity
gene_name_list = [_.upper() for _ in gene_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(BDS_INFO)):
    # check gene_name
    gene_name = adjust_gene_name(BDS_INFO.loc[i, "gene_name"], gene_name_list)
    spe_ori= BDS_INFO.loc[i, "organism"]
    if gene_name.upper() not in gene_name_list: 
        BDS_INFO.loc[i, "wanted"] = False
        if verbose: print(f"{gene_name} not in gene list.")
    else:
        try: gene_name_list_out.remove(gene_name)
        except: pass

    # check DNA or mRNA type
    if BDS_INFO.loc[i, "wanted"] == True:
        if BDS_INFO.loc[i, "mol_type"] != "protein_coding":
            # BDS_INFO.loc[i, "wanted"] = False
            if verbose: print("{} is {}.".format(gene_name, BDS_INFO.loc[i, "mol_type"]))

    # check gene_organism name
    if BDS_INFO.loc[i, "wanted"] == True:
        descrip = BDS_INFO.loc[i, "align_descrip"]
        if pd.isnull(descrip):
            BDS_INFO.loc[i, "wanted"] = False
            if verbose: print(f"{gene_name} not found in BLAST.")
        else:
            descrip = descrip.split("|")
            for des in descrip:
                if gene_name not in des and spe_ori in des:
                    BDS_INFO.loc[i, "wanted"] = False
                    if verbose: print(f"{gene_name} not specific.")
                    break

    # check plus/minus
    if BDS_INFO.loc[i, "wanted"] == True:
        pm_list = BDS_INFO.loc[i, "plus/minus"].split(",")
        if "-1" not in pm_list:
            BDS_INFO.loc[i, "wanted"] = False
            if verbose: print(f"{gene_name} not plus/minus.")

# write the whole information of interest to a excel file in tmp dir
BDS_INFO.to_csv(os.path.join(output, "probes_candidates.csv"))

out_tmp = BDS_INFO[BDS_INFO["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_wanted = list(out_tmp[out_tmp.gene_name == gene]["pos"])
    pos_best = optimize_subsequence(pos_wanted, length=8, min_gap=40, better_gap=80, gene=gene)
    pos_output = out_tmp[out_tmp.gene_name == gene]
    pos_output = pos_output[pos_output["pos"].isin(pos_best)]
    output_df = pd.concat([output_df, pos_output])

# write the output to a xlsx file
output_df.to_csv(os.path.join(output, "probes_wanted.csv"))

Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Oprm1 is nonsense_mediated_decay.
Gene Aif1-204: Not enough pos for 8 binding sites.
Gene Aif1-204: condition too harsh, loose to get better results
[115, 116, 117, 118, 119]
Gene Cartpt-202: Not enough pos for 8 binding sites.
Gene Cartpt-202: condition too harsh, loose to get better results
[40, 41, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 144, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 163, 164, 165, 210, 211, 212, 213, 214, 2

In [39]:
resultsdir = os.path.join(workdir, 'results')
print(resultsdir)

probe/20250612/results


In [40]:
# 去掉重复的probe
resultsdir = os.path.join(workdir, 'results')
def adjust_gene_name(gene_name, gene_list):
    gene_list = [x.upper() for x in gene_list]
    match = re.search(r'(.+)-(\d+)$', gene_name)
    if match:
        base_gene_name = match.group(1)
        if base_gene_name.upper() in gene_list or gene_name.upper() in gene_list: return base_gene_name
        else: return gene_name
    else: return gene_name


result = pd.DataFrame()
for dir in os.listdir(os.path.join(resultsdir)):
    try: result = pd.concat([result, pd.read_csv(os.path.join(resultsdir, dir, "probes_wanted.csv"), index_col=0)])
    except: continue
result["gene_name"] = [adjust_gene_name(_, gene_list) for _ in result["gene_name"]]
result = result.sort_values(["gene_name", "pos"])
# result.drop_duplicates(subset=["bds"], keep="first", inplace=True)
result.drop_duplicates(subset=["plp_bds"], keep="first", inplace=True)
result.head()

,accession,gene_name,mol_type,organism,pos,plp_bds,plp_Tm,plp_bds3',plp_bds5',plp_Tm3',plp_Tm5',mfe,wanted,align_num,align_accession,align_descrip,plus/minus
125,ENSMUST00000173106,Aif1,protein_coding,mouse,115,GCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCAT,67.61,GCCTTGAAGGCTTCAAGTTT,GGACGGCAGATCCTCATCAT,50.65,54.32,-7.1,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
126,ENSMUST00000173106,Aif1,protein_coding,mouse,116,CGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCA,67.70,CGCCTTGAAGGCTTCAAGTT,TGGACGGCAGATCCTCATCA,51.30,54.28,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
127,ENSMUST00000173106,Aif1,protein_coding,mouse,117,GCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATC,69.14,GCGCCTTGAAGGCTTCAAGT,TTGGACGGCAGATCCTCATC,54.67,53.35,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
128,ENSMUST00000173106,Aif1,protein_coding,mouse,118,GGCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCAT,69.61,GGCGCCTTGAAGGCTTCAAG,TTTGGACGGCAGATCCTCAT,54.35,52.49,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
129,ENSMUST00000173106,Aif1,protein_coding,mouse,119,GGGCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCA,70.82,GGGCGCCTTGAAGGCTTCAA,GTTTGGACGGCAGATCCTCA,56.51,55.22,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."


In [41]:
workdir

PosixPath('probe/20250612')

In [42]:
result.to_csv(os.path.join(workdir, 'gene_binding_site.csv'))

result["gene_name"] = result["gene_name"].str.upper()
to_search = [gene for gene in gene_list if len(result[result["gene_name"] == gene.upper()]) < 1]
with open(os.path.join(workdir, "to_search.txt"), "w") as f:
    for line in to_search: f.write(line + "\n")

In [78]:
## 导入模块和作者自己设置的函数
from seqfold import dg, fold, dot_bracket
import random

from Bio.Blast import NCBIXML


def random_seq_list(length=20, num=100):
    nucleotides = ['A', 'T', 'C', 'G']
    sequence = [''.join(random.choice(nucleotides) for _ in range(length)) for j in range(num)]
    return sequence


def hum_dis(seq1, seq2):
    if len(seq1) != len(seq2):
        print("seq_not_match")
    else:
        cont = 0
        for char in range(len(seq1)):
            if seq1[char] != seq2[char]:
                cont += 1
        return cont


def dna_sec_struct(seq, temp=45):
    # Predict the minimum free energy
    mfe = dg(seq, temp=temp)
    # `fold` returns a list of `seqfold.Struct` from the minimum free energy structure
    structs = fold(seq, temp=temp)
    return mfe, structs


# def thre_by_blast(file="./JXAKR9US016-Alignment.xml", thre=18):
#     pos = []
#     with open(file, "r") as blast_output:
#         blast_records = NCBIXML.parse(blast_output)
#         for blast_record in blast_records:
#             save = True
#             for alignment in blast_record.alignments:
#                 # print("Alignment title:", alignment.title)
#                 # print("Length of the alignment:", alignment.length)

#                 # # Iterate over the high-scoring pairs (HSPs) in the alignment
#                 for hsp in alignment.hsps:
#                     # print("HSP score:", hsp.score)
#                     if hsp.score >= thre:
#                         pos.append(False)
#                         save = False
#                         break
#                     # print("HSP bits:", hsp.bits)
#                     # print("HSP query sequence:", hsp.query)
#                     # print("HSP match sequence:", hsp.match)
#                     # print("HSP subject sequence:", hsp.sbjct)
#             if save:
#                 pos.append(True)
#     return pos

def thre_by_blast(file="./JXAKR9US016-Alignment.xml", thre=18):
    """优化版筛选函数：确保每个序列只有一个判断结果"""
    pos = []  # 保存筛选结果的列表
    
    with open(file, "r") as blast_output:
        blast_records = NCBIXML.parse(blast_output)
        
        # 遍历每个BLAST结果记录（每个记录对应一个序列）
        for blast_record in blast_records:
            found_high_score = False  # 标记是否找到高匹配
            
            # 检查每个alignment
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    # 发现高匹配：标记并跳出内层循环
                    if hsp.score >= thre:
                        found_high_score = True
                        break
                
                # 发现高匹配：跳出alignment循环
                if found_high_score:
                    break
                    
            # 将最终判断添加到结果列表（一个序列只有一个结果）
            pos.append(not found_high_score)  # True表示合格序列
            
    return pos

In [ ]:
seq_list = random_seq_list()

seq_list_export = []
for seq in seq_list:
    seq = seq.upper()

    # GGGGG
    if "GGGGG" in seq:
        print(f"{seq}: \tthre_by_G")
        continue

    # dif
    dif = True
    for tmp_seq in seq_list_export:
        if hum_dis(tmp_seq, seq) < 10:
            dif = False
            print(f"{seq}: \tthre_by_dif")
    if not dif:
        continue

    # secondary structure
    mfe, structs = dna_sec_struct(seq, temp=45)
    if mfe < 0:
        print(f"{seq}: \tthre_by_stru\t", dot_bracket(seq, structs))
        continue

    seq_list_export.append(seq)

print(f'{len(seq_list_export)} seqs remained')

In [81]:
## 保存文件,注意查看文件的输出位置
with open(os.path.join(output,"random_seq_filtered.txt"), "w") as f:
    for _ in range(len(seq_list_export)):
        f.write(f'>seq{_}\n' + seq_list_export[_] + "\n")

In [98]:
## 这个步骤主要是选出那些匹配程度较低的序列
## 这样特意设计的序列估计是不不容易自己和自己形成环
## 注意阈值的设置
# pos = thre_by_blast(file='./1KDHC5BG013-Alignment.xml', thre=18)
# pos = thre_by_blast(file=os.path.join(output,'4TMX6X8M016-Alignment.xml'), thre=19)
# sum(pos)

25

In [99]:
## 保存筛选之后的随机序列 
# with open(os.path.join(output,"./random_seq_filtered.txt"), "w") as f:
#     cont = 0
#     for _ in range(len(seq_list_export)):
#         if pos[_]:
#             f.write(f">seq{cont}\n" + seq_list_export[_] + "\n")
#             cont += 1

## 获取random_seq_filtered.txt 感觉可以用一辈子

In [102]:
# 使用优化后的函数
pos = thre_by_blast(file=os.path.join(output, '4TMX6X8M016-Alignment.xml'), thre=19)

# 验证筛选结果
print(f"通过筛选序列数: {sum(pos)}")
print(f"总序列数: {len(pos)}")  # 应该等于70

# 保存符合要求的序列
with open(os.path.join(output, "./random_seq_filtered.txt"), "w") as f:
    cont = 0
    for i, seq in enumerate(seq_list_export):
        if i < len(pos) and pos[i]:  # 添加保护措施
            f.write(f">seq{cont}\n{seq}\n")
            cont += 1

通过筛选序列数: 25
总序列数: 70


## 生成颜色 注意颜色的修改 这次不要绿色

In [105]:
## 根据随机序列生成barcode序列
## 设置运行环境
## 设置函数
import pandas as pd

def hum_dis(seq1, seq2):
    seq1 = seq1.upper()
    seq2 = seq2.upper()
    if len(seq1) != len(seq2):
        # print("seq_not_match")
        # print(seq1, seq2)
        return -1
    else:
        cont = 0
        for char in range(len(seq1)):
            if seq1[char] != seq2[char]:
                cont += 1
        return cont
    

def create_seq_lib(
    seq_list,
    color_fraction={
        "Red": [_ / 4 for _ in range(5)],
        # "Green": [_ / 2 for _ in range(3)],
        "Blue": [_ / 4 for _ in range(5)],
        "Yellow": [_ / 4 for _ in range(5)],
    },
):
    seq_lib = pd.DataFrame(columns=["seq", "color", "grade", "fraction"])
    color_list = []
    for color in color_fraction.keys():
        color_list += [color] * len(color_fraction[color])
    seq_lib["color"] = color_list

    if len(seq_lib["color"]) > len(seq_list):
        print("Seq Not Enough")
        return ValueError

    seq_lib["seq"] = seq_list[: len(seq_lib["color"])]

    fra = []
    grade = []
    for color in color_fraction.keys():
        fra += color_fraction[color]
        grade += [_ for _ in range(len(color_fraction[color]))]
    seq_lib["fraction"] = fra
    seq_lib["grade"] = grade

    return seq_lib


import itertools

def create_barcode_lib(
    seq_lib,
    color_order=["Green", "Red", "Blue", "Yellow"],
    sum_num=5,
    sum_list=["Red", "Yellow", "Blue"],
):
    color_order_seq = [_ + "seq" for _ in color_order]
    barcode_lib = pd.DataFrame(columns=["barcode"] + color_order + color_order_seq)

    grade_list = {
        color: list(seq_lib[seq_lib.color == color].grade.unique())
        for color in color_order
    }
    barcode_lib[color_order] = list(itertools.product(*grade_list.values()))

    for i in range(len(barcode_lib)):
        grade = barcode_lib.loc[i, color_order]

        barcode_sub_list = [
            list(
                seq_lib[
                    (seq_lib.color == color_order[_]) & (seq_lib.grade == grade[_])
                ].seq
            )[0]
            for _ in range(len(color_order))
        ]
        barcode_lib.loc[i, color_order_seq] = barcode_sub_list
        barcode_lib.loc[i, "barcode"] = "".join(barcode_sub_list)

    if sum_num:
        barcode_lib["sum"] = barcode_lib[sum_list].sum(axis=1)
        barcode_lib = barcode_lib[
            # (barcode_lib["sum"] >= 1) 
            # &
            (barcode_lib["sum"] == sum_num)
        ]
        barcode_lib.set_index('barcode', inplace=True)

    return barcode_lib

In [104]:
## 读取上个random_seq_filtered.txt文件,可以用很久的颜色序列
file =os.path.join(output,'random_seq_filtered.txt')
with open(file, "r") as f:
    seq_list = f.readlines()
seq_list = [seq_list[_].replace('\n','') for _ in range(1,len(seq_list),2)]
print(seq_list)

['ACAAAAACGATCGCTACTTA', 'CCCTAATGATTGAATGCGTG', 'CCTGCCAAGCATAGATGATC', 'TGGTATTTGGTACGAATTGC', 'CGTGCTCGACTACGGCTGAC', 'TGGATTATAATGTGGGACCT', 'TGAGCGTCTGCAACCAGGCT', 'CTAGGCCCAACTTATAAACA', 'AAATGCGCCGAATGTACCAA', 'CCCTAAGATCTCCGCTCCCA', 'TAAGCACGGGTCACGAACGT', 'GATTCTCAGGTGGGGTTGCT', 'CACCGTTTCGCTTGAGAAGT', 'ACGTATGCGACACGATTCCG', 'GATGGTTCTTTAGCATAGAA', 'GTAGCGATAGTACACCGAGC', 'TGATTCGTGTCCCGGCTCTA', 'TTTCAGAGGGGTCCCAAGCC', 'AGATCCACGCCTCTAAGCCA', 'TTCTACATCATACCCTTACA', 'TCGGTAGGACTCCTTGTATA', 'ACGGTTTAGGCGAGGAGATT', 'GGTAGTTATATAGAAACCTT', 'CAACTCCCGCACGTTACTAA', 'GATGTATGTAGGCCCGAATG']


In [107]:
## 根据上面的序列生成矩阵
## 在这个步骤里面 如果提供的随机的序列不够 会报错 因为这些序列分不过来
## 但如果本身并不需要那么多荧光颜色的话 这边生成多少种类颜色的代码可以修改
## 此代码可根据需要进行修改
seq_lib = create_seq_lib(
    seq_list=seq_list,
    color_fraction={
        # "Green": [0, 0.5, 1],
        "Red": [_ / 4 for _ in range(5)],
        "Blue": [_ / 4 for _ in range(5)],
        "Yellow": [_ / 4 for _ in range(5)],
    },
)
seq_lib

,seq,color,grade,fraction
0,ACAAAAACGATCGCTACTTA,Red,0,0.00
1,CCCTAATGATTGAATGCGTG,Red,1,0.25
2,CCTGCCAAGCATAGATGATC,Red,2,0.50
3,TGGTATTTGGTACGAATTGC,Red,3,0.75
4,CGTGCTCGACTACGGCTGAC,Red,4,1.00
5,TGGATTATAATGTGGGACCT,Blue,0,0.00
6,TGAGCGTCTGCAACCAGGCT,Blue,1,0.25
7,CTAGGCCCAACTTATAAACA,Blue,2,0.50
8,AAATGCGCCGAATGTACCAA,Blue,3,0.75
9,CCCTAAGATCTCCGCTCCCA,Blue,4,1.00


In [109]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
barcode_lib = create_barcode_lib(
    seq_lib,
    # color_order=["Green", "Red", "Blue", "Yellow"],
    color_order=["Red", "Blue", "Yellow"],
    sum_num=5,
    sum_list=["Red", "Yellow", "Blue"],
)
barcode_lib

,Red,Blue,Yellow,Redseq,Blueseq,Yellowseq,sum
barcode,,,,,,,
ACAAAAACGATCGCTACTTATGAGCGTCTGCAACCAGGCTGATGGTTCTTTAGCATAGAA,0,1,4,ACAAAAACGATCGCTACTTA,TGAGCGTCTGCAACCAGGCT,GATGGTTCTTTAGCATAGAA,5
ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTATGCGACACGATTCCG,0,2,3,ACAAAAACGATCGCTACTTA,CTAGGCCCAACTTATAAACA,ACGTATGCGACACGATTCCG,5
ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGTTTCGCTTGAGAAGT,0,3,2,ACAAAAACGATCGCTACTTA,AAATGCGCCGAATGTACCAA,CACCGTTTCGCTTGAGAAGT,5
ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCTCAGGTGGGGTTGCT,0,4,1,ACAAAAACGATCGCTACTTA,CCCTAAGATCTCCGCTCCCA,GATTCTCAGGTGGGGTTGCT,5
CCCTAATGATTGAATGCGTGTGGATTATAATGTGGGACCTGATGGTTCTTTAGCATAGAA,1,0,4,CCCTAATGATTGAATGCGTG,TGGATTATAATGTGGGACCT,GATGGTTCTTTAGCATAGAA,5
CCCTAATGATTGAATGCGTGTGAGCGTCTGCAACCAGGCTACGTATGCGACACGATTCCG,1,1,3,CCCTAATGATTGAATGCGTG,TGAGCGTCTGCAACCAGGCT,ACGTATGCGACACGATTCCG,5
CCCTAATGATTGAATGCGTGCTAGGCCCAACTTATAAACACACCGTTTCGCTTGAGAAGT,1,2,2,CCCTAATGATTGAATGCGTG,CTAGGCCCAACTTATAAACA,CACCGTTTCGCTTGAGAAGT,5
CCCTAATGATTGAATGCGTGAAATGCGCCGAATGTACCAAGATTCTCAGGTGGGGTTGCT,1,3,1,CCCTAATGATTGAATGCGTG,AAATGCGCCGAATGTACCAA,GATTCTCAGGTGGGGTTGCT,5
CCCTAATGATTGAATGCGTGCCCTAAGATCTCCGCTCCCATAAGCACGGGTCACGAACGT,1,4,0,CCCTAATGATTGAATGCGTG,CCCTAAGATCTCCGCTCCCA,TAAGCACGGGTCACGAACGT,5


In [111]:
## 保存seq 保存生成的barcode
# #tmp=os.getcwd()
# seq_lib.to_csv(r"./seq_lib.csv")
# barcode_lib.to_csv(r"./barcode_lib.csv")

## 保存seq 保存生成的barcode
#tmp=os.getcwd()
seq_lib.to_csv(os.path.join(output,'seq_lib.csv'))
barcode_lib.to_csv(os.path.join(output,'barcode_lib.csv'))

In [112]:
binding_df = pd.read_csv(DATASET_DIR / RUNID / "gene_binding_site.csv")
if organism == 'mouse': binding_df['gene_name'] = binding_df['gene_name'].str.capitalize()
elif organism == 'human': binding_df['gene_name'] = binding_df['gene_name'].str.upper()
# binding_df = binding_df[binding_df['gene_name'].isin(gene_list)]
print(len(binding_df))
binding_df.head()

207


,Unnamed: 0,accession,gene_name,mol_type,organism,pos,plp_bds,plp_Tm,plp_bds3',plp_bds5',plp_Tm3',plp_Tm5',mfe,wanted,align_num,align_accession,align_descrip,plus/minus
0,125,ENSMUST00000173106,Aif1,protein_coding,mouse,115,GCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCAT,67.61,GCCTTGAAGGCTTCAAGTTT,GGACGGCAGATCCTCATCAT,50.65,54.32,-7.1,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
1,126,ENSMUST00000173106,Aif1,protein_coding,mouse,116,CGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCA,67.70,CGCCTTGAAGGCTTCAAGTT,TGGACGGCAGATCCTCATCA,51.30,54.28,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
2,127,ENSMUST00000173106,Aif1,protein_coding,mouse,117,GCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATC,69.14,GCGCCTTGAAGGCTTCAAGT,TTGGACGGCAGATCCTCATC,54.67,53.35,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
3,128,ENSMUST00000173106,Aif1,protein_coding,mouse,118,GGCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCAT,69.61,GGCGCCTTGAAGGCTTCAAG,TTTGGACGGCAGATCCTCAT,54.35,52.49,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."
4,129,ENSMUST00000173106,Aif1,protein_coding,mouse,119,GGGCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCA,70.82,GGGCGCCTTGAAGGCTTCAA,GTTTGGACGGCAGATCCTCA,56.51,55.22,-7.3,True,25,AK006184.1|U82792.1|NM_019467.4|JN957732.1|NM_...,"Mus musculus adult male testis cDNA, RIKEN fu...","-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-..."


In [113]:
## 告诉计算机这次用是那篇文章涉及的分析 
## 其实可以修改这个if函数 只要最终输出的是barcode_df 这样一个矩阵
## 先尝试将barcode列选出来
PANEL = 'PRISM'
if PANEL == 'PRISM':
    probe_df = pd.DataFrame()
    barcode_df = pd.read_csv(os.path.join(output,'barcode_lib.csv'))
elif PANEL == 'SPRINTseq':
    barcode_df = pd.read_excel(DATASET_DIR / "SPRINTSEQ_369_barcode.xlsx", index_col=0)[['Barcode sequence']]
    primer_l = 'TCCCTACACGACGCTCTTCCGATCT'
    primer_r = 'CATTCCTGCTGAACCGCTCTTCCGA'
    barcode_df['Barcode(70bp)'] = primer_l + barcode_df['Barcode sequence'] + primer_r + barcode_df['Barcode sequence']
barcode_df.head()

,barcode,Red,Blue,Yellow,Redseq,Blueseq,Yellowseq,sum
0,ACAAAAACGATCGCTACTTATGAGCGTCTGCAACCAGGCTGATGGT...,0,1,4,ACAAAAACGATCGCTACTTA,TGAGCGTCTGCAACCAGGCT,GATGGTTCTTTAGCATAGAA,5
1,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,0,2,3,ACAAAAACGATCGCTACTTA,CTAGGCCCAACTTATAAACA,ACGTATGCGACACGATTCCG,5
2,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,0,3,2,ACAAAAACGATCGCTACTTA,AAATGCGCCGAATGTACCAA,CACCGTTTCGCTTGAGAAGT,5
3,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,0,4,1,ACAAAAACGATCGCTACTTA,CCCTAAGATCTCCGCTCCCA,GATTCTCAGGTGGGGTTGCT,5
4,CCCTAATGATTGAATGCGTGTGGATTATAATGTGGGACCTGATGGT...,1,0,4,CCCTAATGATTGAATGCGTG,TGGATTATAATGTGGGACCT,GATGGTTCTTTAGCATAGAA,5


In [117]:
## 每个基因保存3条序列
## 设置一些空参数
probe_df = pd.DataFrame()
cont = 0
prism_pos = 0
prism_pos_list = [_+1 for _ in range(15)]
prism = prism_pos_list[prism_pos]
max_cont = 3
pre_gene_name = binding_df["gene_name"].iloc[0]
for num, gene in enumerate(binding_df["gene_name"]):
    if pre_gene_name != gene:
        pre_gene_name = gene
        cont = 0
        prism_pos += 1
        prism = prism_pos_list[prism_pos]
    elif cont == max_cont:
        continue
    # print(num, gene, prism)
    cont += 1
    binding = binding_df["plp_bds"].iloc[num]
    assert len(binding) == 40, f"binding site at pos {num} length is not 40bp: {binding}, {len(binding)} instead."

    binding_l = binding[:20].lower()
    binding_r = binding[20:].lower()
    barcode = barcode_df.loc[prism, "barcode"]
    probe = binding_r + barcode + binding_l

    if PANEL == 'PRISM':
        probe_info = pd.DataFrame({
            "PRISM": [f"PRISM_{prism}"],
            "gene":[f'{gene}_{cont}'],
            "probe_name":[f'PR_{prism}_{gene}_{cont}'],
            "probe_seq": [probe],
            "barcode_seq": [barcode],
            "binding_seq": [binding],})

    elif PANEL == 'SPRINTseq':
        probe_info = pd.DataFrame({
            "SPRINTseq": [f"SPRINTseq_{prism}"],
            "gene":[f'{gene}'],
            "probe_name":[f'Seq_{prism}_{gene}_{cont}'],
            "probe": [probe],
            "barcode": [barcode],
            "binding": [binding],})
    if len(probe_df) == 0: probe_df = probe_info
    else: probe_df = pd.concat([probe_df, probe_info])
probe_df = probe_df.reset_index(drop=True)
probe_df

,PRISM,gene,probe_name,probe_seq,barcode_seq,binding_seq
0,PRISM_1,Aif1_1,PR_1_Aif1_1,ggacggcagatcctcatcatACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCAT
1,PRISM_1,Aif1_2,PR_1_Aif1_2,tggacggcagatcctcatcaACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,CGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCA
2,PRISM_1,Aif1_3,PR_1_Aif1_3,ttggacggcagatcctcatcACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATC
3,PRISM_2,Cartpt_1,PR_2_Cartpt_1,tagcagtagcagcagggcggACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCGG
4,PRISM_2,Cartpt_2,PR_2_Cartpt_2,gtagcagtagcagcagggcgACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GGCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCG
5,PRISM_2,Cartpt_3,PR_2_Cartpt_3,tagatgtccagggctcggggACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,ACGCATCATCCACGGCAGAGTAGATGTCCAGGGCTCGGGG
6,PRISM_3,Flpo_lilab_1,PR_3_Flpo_lilab_1,cgaactgccgcaccagcaccACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,GGGCCTCTCGAATCTCTCCACGAACTGCCGCACCAGCACC
7,PRISM_3,Flpo_lilab_2,PR_3_Flpo_lilab_2,cgaagctcaggctgttgctgACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,CAGGCTCTTGTTCACGATGTCGAAGCTCAGGCTGTTGCTG
8,PRISM_3,Flpo_lilab_3,PR_3_Flpo_lilab_3,cggtgatgtcgctctggtgcACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,CTGCAGGCTGGACACGATGTCGGTGATGTCGCTCTGGTGC
9,PRISM_4,Gad1_1,PR_4_Gad1_1,ttcgaggaggttgcaggcgaCCCTAATGATTGAATGCGTGTGGATT...,CCCTAATGATTGAATGCGTGTGGATTATAATGTGGGACCTGATGGT...,TATTAGGATCCGCTCCCGCGTTCGAGGAGGTTGCAGGCGA


In [127]:
path=os.path.join(output,'./')
probe_df.to_csv( path+f'{PANEL}_probe.csv')
print(len(probe_df))
probe_df.head()

45


,PRISM,gene,probe_name,probe_seq,barcode_seq,binding_seq
0,PRISM_1,Aif1_1,PR_1_Aif1_1,ggacggcagatcctcatcatACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCAT
1,PRISM_1,Aif1_2,PR_1_Aif1_2,tggacggcagatcctcatcaACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,CGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCA
2,PRISM_1,Aif1_3,PR_1_Aif1_3,ttggacggcagatcctcatcACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATC
3,PRISM_2,Cartpt_1,PR_2_Cartpt_1,tagcagtagcagcagggcggACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCGG
4,PRISM_2,Cartpt_2,PR_2_Cartpt_2,gtagcagtagcagcagggcgACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GGCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCG


In [128]:
# 已经获取序列 
# 对序列进行检查
def overlap_degree(str1, str2):
    max_overlap = min(len(str1), len(str2))
    
    for i in range(max_overlap, 0, -1):
        if str1[-i:] == str2[:i]:
            return i
    
    return 0
meta = pd.read_csv(os.path.join(output,'PRISM_probe.csv'),index_col=0)
meta

,PRISM,gene,probe_name,probe_seq,barcode_seq,binding_seq
0,PRISM_1,Aif1_1,PR_1_Aif1_1,ggacggcagatcctcatcatACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCAT
1,PRISM_1,Aif1_2,PR_1_Aif1_2,tggacggcagatcctcatcaACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,CGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATCA
2,PRISM_1,Aif1_3,PR_1_Aif1_3,ttggacggcagatcctcatcACAAAAACGATCGCTACTTACTAGGC...,ACAAAAACGATCGCTACTTACTAGGCCCAACTTATAAACAACGTAT...,GCGCCTTGAAGGCTTCAAGTTTGGACGGCAGATCCTCATC
3,PRISM_2,Cartpt_1,PR_2_Cartpt_1,tagcagtagcagcagggcggACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCGG
4,PRISM_2,Cartpt_2,PR_2_Cartpt_2,gtagcagtagcagcagggcgACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,GGCACGGGCACCCAGCAAAGGTAGCAGTAGCAGCAGGGCG
5,PRISM_2,Cartpt_3,PR_2_Cartpt_3,tagatgtccagggctcggggACAAAAACGATCGCTACTTAAAATGC...,ACAAAAACGATCGCTACTTAAAATGCGCCGAATGTACCAACACCGT...,ACGCATCATCCACGGCAGAGTAGATGTCCAGGGCTCGGGG
6,PRISM_3,Flpo_lilab_1,PR_3_Flpo_lilab_1,cgaactgccgcaccagcaccACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,GGGCCTCTCGAATCTCTCCACGAACTGCCGCACCAGCACC
7,PRISM_3,Flpo_lilab_2,PR_3_Flpo_lilab_2,cgaagctcaggctgttgctgACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,CAGGCTCTTGTTCACGATGTCGAAGCTCAGGCTGTTGCTG
8,PRISM_3,Flpo_lilab_3,PR_3_Flpo_lilab_3,cggtgatgtcgctctggtgcACAAAAACGATCGCTACTTACCCTAA...,ACAAAAACGATCGCTACTTACCCTAAGATCTCCGCTCCCAGATTCT...,CTGCAGGCTGGACACGATGTCGGTGATGTCGCTCTGGTGC
9,PRISM_4,Gad1_1,PR_4_Gad1_1,ttcgaggaggttgcaggcgaCCCTAATGATTGAATGCGTGTGGATT...,CCCTAATGATTGAATGCGTGTGGATTATAATGTGGGACCTGATGGT...,TATTAGGATCCGCTCCCGCGTTCGAGGAGGTTGCAGGCGA


## 

In [129]:
# 计算序列存在重合的情况 避免形成圆圈
# 输出的这些都是不能被使用的
for prism in range(15): ## 多少个基因这边输入多少
    detect_seq = list(meta[meta['PRISM'] == f'PRISM_{prism+1}']['binding_seq'])
    gene = list(meta[meta['PRISM'] == f'PRISM_{prism+1}']['probe_name'])
    gene = gene[0]
    for i in range(-1,2,1):
        degree = max(overlap_degree(detect_seq[i], detect_seq[i+1]), overlap_degree(detect_seq[i+1], detect_seq[i]))
        if degree > 5:
            print(f'{gene}, {i}, {i+1}, degree={degree}')

PR_1_Aif1_1, -1, 0, degree=38
PR_1_Aif1_1, 0, 1, degree=39
PR_1_Aif1_1, 1, 2, degree=39
PR_2_Cartpt_1, 0, 1, degree=39
PR_4_Gad1_1, -1, 0, degree=38
PR_4_Gad1_1, 0, 1, degree=39
PR_4_Gad1_1, 1, 2, degree=39
PR_5_Gapdh_1, -1, 0, degree=36
PR_5_Gapdh_1, 0, 1, degree=39
PR_5_Gapdh_1, 1, 2, degree=37
PR_9_Oprm1_1, -1, 0, degree=34
PR_9_Oprm1_1, 0, 1, degree=35
PR_9_Oprm1_1, 1, 2, degree=39
PR_10_Pdyn_1, -1, 0, degree=38
PR_10_Pdyn_1, 0, 1, degree=39
PR_10_Pdyn_1, 1, 2, degree=39
PR_13_Sst_1, -1, 0, degree=38
PR_13_Sst_1, 0, 1, degree=39
PR_13_Sst_1, 1, 2, degree=39
PR_14_Vipr2_1, -1, 0, degree=36
PR_14_Vipr2_1, 0, 1, degree=37
PR_14_Vipr2_1, 1, 2, degree=39


In [130]:
import pandas as pd
def dna_sec_struct(seq, temp=45):
    # Predict the minimum free energy
    mfe = dg(seq, temp=temp)
    # `fold` returns a list of `seqfold.Struct` from the minimum free energy structure
    structs = fold(seq, temp=temp)
    return mfe, structs

def seq_minus(seq):
    translib = {"A": "T", "T": "A", "C": "G", "G": "C"}
    return "".join(list(reversed([translib[i] for i in seq])))

In [ ]:
# 注意提供的序列并不是全部的序列，需要重新琢磨以下这个代码看这个具体的判断标准究竟是什么。
# 可能要用binding seq也说不定
# padlock = pd.read_excel(r'E:\TMC\probe_designer\dataset\2024.3.16_TCR&mutation_3_Breast_cancer\binding_site_revised.xlsx', sheet_name='padlock')
padlock = pd.read_csv('./PRISM_probe.csv',index_col=0)[['probe_name','probe_seq']]
padlock

In [132]:
## 将probe_seq列小写全部换成大写
# 将 probe_seq 列中的小写字母转换为大写
padlock["probe_seq"] = padlock["probe_seq"].str.upper()
# 验证结果
print(padlock.head())

      probe_name                                          probe_seq
0    PR_1_Aif1_1  GGACGGCAGATCCTCATCATACAAAAACGATCGCTACTTACTAGGC...
1    PR_1_Aif1_2  TGGACGGCAGATCCTCATCAACAAAAACGATCGCTACTTACTAGGC...
2    PR_1_Aif1_3  TTGGACGGCAGATCCTCATCACAAAAACGATCGCTACTTACTAGGC...
3  PR_2_Cartpt_1  TAGCAGTAGCAGCAGGGCGGACAAAAACGATCGCTACTTAAAATGC...
4  PR_2_Cartpt_2  GTAGCAGTAGCAGCAGGGCGACAAAAACGATCGCTACTTAAAATGC...


In [133]:
## 保存最终的文件
# padlock.to_csv('./probe_seq.csv')
padlock.to_csv(os.path.join(output,'probe_seq.csv'))

In [134]:
## 调用上面的函数 查看评分
for index, (name, seq) in padlock.iterrows():
    seq = seq_minus(seq)
    seq = seq.upper() * 3
    mfe, structs = dna_sec_struct(seq, temp=60)
    # if mfe < 0:
    print(f"{name}:{mfe}\n{seq}\n{dot_bracket(seq, structs)}")
    # continue

PR_1_Aif1_1:-3.4
AAACTTGAAGCCTTCAAGGCCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTATGATGAGGATCTGCCGTCCAAACTTGAAGCCTTCAAGGCCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTATGATGAGGATCTGCCGTCCAAACTTGAAGCCTTCAAGGCCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTATGATGAGGATCTGCCGTCC
..............................................................................................(((.((....((((((.((((((((.(((.......................................................................))).))....)))))).)))))))).))).............................................................................
PR_1_Aif1_2:-2.0
AACTTGAAGCCTTCAAGGCGCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTTGATGAGGATCTGCCGTCCAAACTTGAAGCCTTCAAGGCGCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTTGATGAGGATCTGCCGTCCAAACTTGAAGCCTTCAAGGCGCGGAATCGTGTCGCATACGTTGTTTATAAGTTGGGCCTAGTAAGTAGCGATCGTTTTTGTTGATGAGGATCTGCCGTCCA
...............................................................